In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Groupby한 Series에서 4, 5 act_label 널값 처리를 못 하고 있다.

act = pd.read_csv('stackPerDevice.csv')
act.shape



(71957, 9)

In [158]:
ABSENCE = 0 # 부재
NOT_ACT = 1 # 미활동
ACT = 2 # 활동
CAUTION = 3 #주의
FALL = 4 # 낙상


def getEmergencyLabel(status, defaultStatus):
    if status == "fall_detected" or status == "fall_exit" or status == "fall_confirmed":
        return CAUTION
    elif status == "calling" or status == "finished":
        return FALL
    else:
        return defaultStatus

def makeStatus(df):
#     if df['avg_people_cnt'] == 0: #없는 경우가 있다.
#         return ABSENCE
#     else:
#     act_label = ABSENCE
    if df['avg_activity'] == 0:
        act_label = ABSENCE
    elif df['avg_activity'] < 3.0 or df['max_activity'] < 20.0:
        act_label = NOT_ACT
    else:
        act_label = ACT
        
    act_label = getEmergencyLabel(df['lastStatus'], act_label)
    return act_label

In [152]:
newact = act.copy()
newact['act_label'] = newact.apply(makeStatus, axis=1)
newact

,deviceId,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus,act_label
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769933,0,0.0,14.586480,32.6205,5,NaN,NaN,3
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769934,0,0.0,4.635100,9.6501,4,NaN,NaN,1
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769936,0,0.0,5.351220,40.8439,10,NaN,NaN,3
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769937,0,0.0,1.761400,4.7079,3,NaN,NaN,1
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769938,0,0.0,4.397413,18.2786,8,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
71952,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774215,0,0.0,6.500133,11.0445,3,NaN,NaN,1
71953,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774221,0,0.0,14.533129,61.1274,34,NaN,NaN,3
71954,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774228,0,0.0,12.176412,35.1317,8,NaN,NaN,3
71955,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774229,0,0.0,13.513771,48.3919,14,NaN,NaN,3


In [159]:
#디바이스별로 집계를 구하고, 그 결과를 newdata에 저장함.
newdata = pd.DataFrame()

# group의 키 목록을 구함
grouped = act.groupby('deviceId')

for deviceid in grouped.groups.keys():
    newact = act[ act['deviceId'] == deviceid ].copy()
    newact['act_label'] = newact.apply(makeStatus, axis=1)
    newdata = newdata.append(newact)
    
newdata

C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\3068236804.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newdata = newdata.append(newact)
C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\3068236804.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newdata = newdata.append(newact)
C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\3068236804.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newdata = newdata.append(newact)
C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\3068236804.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newdata = newdata.append(newact)
C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\306823

,deviceId,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus,act_label
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769933,0,0.0,14.586480,32.6205,5,NaN,NaN,2
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769934,0,0.0,4.635100,9.6501,4,NaN,NaN,1
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769936,0,0.0,5.351220,40.8439,10,NaN,NaN,2
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769937,0,0.0,1.761400,4.7079,3,NaN,NaN,1
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769938,0,0.0,4.397413,18.2786,8,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
71952,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774215,0,0.0,6.500133,11.0445,3,NaN,NaN,1
71953,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774221,0,0.0,14.533129,61.1274,34,NaN,NaN,2
71954,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774228,0,0.0,12.176412,35.1317,8,NaN,NaN,2
71955,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774229,0,0.0,13.513771,48.3919,14,NaN,NaN,2


In [160]:
# deviceid, act_label별 집계수를 구한다.
newdata['new_cnt'] = newdata.groupby(['deviceId','act_label'])['act_label'].transform('count')
newdata

,deviceId,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus,act_label,new_cnt
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769933,0,0.0,14.586480,32.6205,5,NaN,NaN,2,664
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769934,0,0.0,4.635100,9.6501,4,NaN,NaN,1,843
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769936,0,0.0,5.351220,40.8439,10,NaN,NaN,2,664
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769937,0,0.0,1.761400,4.7079,3,NaN,NaN,1,843
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,2769938,0,0.0,4.397413,18.2786,8,NaN,NaN,1,843
...,...,...,...,...,...,...,...,...,...,...,...
71952,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774215,0,0.0,6.500133,11.0445,3,NaN,NaN,1,671
71953,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774221,0,0.0,14.533129,61.1274,34,NaN,NaN,2,2153
71954,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774228,0,0.0,12.176412,35.1317,8,NaN,NaN,2,2153
71955,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2774229,0,0.0,13.513771,48.3919,14,NaN,NaN,2,2153


In [161]:
newdata1 = newdata[['deviceId','act_label', 'new_cnt']]
newdata1


,deviceId,act_label,new_cnt
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,2,664
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,1,843
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2,664
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,1,843
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,1,843
...,...,...,...
71952,id_MzQ6QUI6OTU6NzE6QTU6MTQ,1,671
71953,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2,2153
71954,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2,2153
71955,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2,2153


In [219]:
# new1 = newdata1.groupby(['deviceId','act_label']).count()
# new1

In [164]:
gp = new1.groupby('deviceId')

kim = pd.DataFrame(np.zeros((29,3)), columns=['deviceId', 'act_label', 'act_cnt'])


# print(kim.columns)

index = 0 

for id in gp.groups.keys():
#     print(id)
    kim.loc[index] = [id, 0, np.nan]
    index += 1
    kim.loc[index] = [id, 1, np.nan]
    index += 1
    kim.loc[index] = [id, 2, np.nan]
    index += 1
    kim.loc[index] = [id, 3, np.nan]
    index += 1
    kim.loc[index] = [id, 4, np.nan]
    index += 1

kim.act_cnt = np.nan
kim

,deviceId,act_label,act_cnt
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,0.0,NaN
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,1.0,NaN
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2.0,NaN
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,3.0,NaN
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,4.0,NaN
...,...,...,...
140,id_MzQ6QUI6OTU6NzE6QTU6MTQ,0.0,NaN
141,id_MzQ6QUI6OTU6NzE6QTU6MTQ,1.0,NaN
142,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2.0,NaN
143,id_MzQ6QUI6OTU6NzE6QTU6MTQ,3.0,NaN


In [165]:
kim = pd.merge(kim, new1, on=['deviceId', 'act_label'], how="left")
kim

,deviceId,act_label,act_cnt,new_cnt
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,0.0,NaN,13.0
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,1.0,NaN,843.0
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2.0,NaN,664.0
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,3.0,NaN,20.0
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,4.0,NaN,NaN
...,...,...,...,...
140,id_MzQ6QUI6OTU6NzE6QTU6MTQ,0.0,NaN,2.0
141,id_MzQ6QUI6OTU6NzE6QTU6MTQ,1.0,NaN,671.0
142,id_MzQ6QUI6OTU6NzE6QTU6MTQ,2.0,NaN,2153.0
143,id_MzQ6QUI6OTU6NzE6QTU6MTQ,3.0,NaN,60.0


In [166]:
kim['new_cnt'] = kim['new_cnt'].fillna(0)
kim.head(50)

,deviceId,act_label,act_cnt,new_cnt
0,id_MzA6QUU6QTQ6RTI6RTA6NEM,0.0,NaN,13.0
1,id_MzA6QUU6QTQ6RTI6RTA6NEM,1.0,NaN,843.0
2,id_MzA6QUU6QTQ6RTI6RTA6NEM,2.0,NaN,664.0
3,id_MzA6QUU6QTQ6RTI6RTA6NEM,3.0,NaN,20.0
4,id_MzA6QUU6QTQ6RTI6RTA6NEM,4.0,NaN,0.0
5,id_MzA6QUU6QTQ6RTM6OTc6RjQ,0.0,NaN,3.0
6,id_MzA6QUU6QTQ6RTM6OTc6RjQ,1.0,NaN,175.0
7,id_MzA6QUU6QTQ6RTM6OTc6RjQ,2.0,NaN,236.0
8,id_MzA6QUU6QTQ6RTM6OTc6RjQ,3.0,NaN,3.0
9,id_MzA6QUU6QTQ6RTM6OTc6RjQ,4.0,NaN,0.0


In [ ]:
# kim.apply(lambda x : print(x['deviceId']))
# kim.apply(lambda x : print(x['act_label']))

In [216]:
kim2 = pd.DataFrame()
kim2 = kim.pivot('deviceId', 'act_label', 'new_cnt')
kim2

C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\1496693754.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  kim2 = kim.pivot('deviceId', 'act_label', 'new_cnt')


act_label,0.0,1.0,2.0,3.0,4.0
deviceId,,,,,
id_MzA6QUU6QTQ6RTI6RTA6NEM,13.0,843.0,664.0,20.0,0.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ,3.0,175.0,236.0,3.0,0.0
id_MzA6QUU6QTQ6RTM6RTA6M0M,2.0,275.0,1042.0,174.0,2.0
id_MzA6QUU6QTQ6RTM6RTA6MzA,9.0,613.0,2647.0,14.0,0.0
id_MzQ6QUI6OTU6NzE6OTU6NzA,3.0,650.0,2606.0,17.0,1.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg,1.0,784.0,2057.0,36.0,1.0
id_MzQ6QUI6OTU6NzE6OTY6MDA,7.0,429.0,3297.0,0.0,0.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ,11.0,436.0,2445.0,0.0,0.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M,2.0,386.0,1223.0,171.0,0.0


In [203]:
kim2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, id_MzA6QUU6QTQ6RTI6RTA6NEM to id_MzQ6QUI6OTU6NzE6QTU6MTQ
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     29 non-null     float64
 1   1.0     29 non-null     float64
 2   2.0     29 non-null     float64
 3   3.0     29 non-null     float64
 4   4.0     29 non-null     float64
dtypes: float64(5)
memory usage: 1.4+ KB


In [214]:
kim2.apply(lambda row : (4320  - row.iloc[1] - row.iloc[2] - row.iloc[3]- row.iloc[4]) , axis=1 )

deviceId
id_MzA6QUU6QTQ6RTI6RTA6NEM    2793.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ    3906.0
id_MzA6QUU6QTQ6RTM6RTA6M0M    2827.0
id_MzA6QUU6QTQ6RTM6RTA6MzA    1046.0
id_MzQ6QUI6OTU6NzE6OTU6NzA    1046.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg    1442.0
id_MzQ6QUI6OTU6NzE6OTY6MDA     594.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ    1439.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M    2540.0
id_MzQ6QUI6OTU6NzE6OTY6RTQ    2492.0
id_MzQ6QUI6OTU6NzE6OTc6RDQ    2641.0
id_MzQ6QUI6OTU6NzE6OTk6MDg    1637.0
id_MzQ6QUI6OTU6NzE6OUI6MTA    2120.0
id_MzQ6QUI6OTU6NzE6OUM6NDA    1679.0
id_MzQ6QUI6OTU6NzE6OUM6OUM    1172.0
id_MzQ6QUI6OTU6NzE6OUM6Qzg    1327.0
id_MzQ6QUI6OTU6NzE6OUM6REM    2014.0
id_MzQ6QUI6OTU6NzE6OUQ6Mzg    1587.0
id_MzQ6QUI6OTU6NzE6OUQ6NEM    2173.0
id_MzQ6QUI6OTU6NzE6OUU6MzQ    1253.0
id_MzQ6QUI6OTU6NzE6OUU6QTg    1494.0
id_MzQ6QUI6OTU6NzE6OUU6RDA    1982.0
id_MzQ6QUI6OTU6NzE6QTA6ODg    1376.0
id_MzQ6QUI6OTU6NzE6QTE6MDA    1871.0
id_MzQ6QUI6OTU6NzE6QTQ6Mjg    2080.0
id_MzQ6QUI6OTU6NzE6QTQ6Nzg    2374.0
id_MzQ6QUI6OTU6NzE6QTQ6ODg   

In [217]:
# kim2.apply(lambda row : print(row.iloc[0]), axis=1 )
# kim2.loc[:, ['new_0']] = kim2.apply(lambda row : (4320  - row.iloc[1] + row.iloc[2] + row.iloc[3]+ row.iloc[4]) , axis=1 )
kim2.loc[:,['new_0']] = kim2.apply(lambda row : (4320  - row.iloc[1] - row.iloc[2] - row.iloc[3]- row.iloc[4]) , axis=1 )
# kim2.loc[:, [0]] = kim2.apply(lambda row : (4320  - row.iloc[1] + row.iloc[2] + row.iloc[3]+ row.iloc[4] + row.iloc[5]) , axis=1 )
kim2

act_label,0.0,1.0,2.0,3.0,4.0,new_0
deviceId,,,,,,
id_MzA6QUU6QTQ6RTI6RTA6NEM,13.0,843.0,664.0,20.0,0.0,2793.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ,3.0,175.0,236.0,3.0,0.0,3906.0
id_MzA6QUU6QTQ6RTM6RTA6M0M,2.0,275.0,1042.0,174.0,2.0,2827.0
id_MzA6QUU6QTQ6RTM6RTA6MzA,9.0,613.0,2647.0,14.0,0.0,1046.0
id_MzQ6QUI6OTU6NzE6OTU6NzA,3.0,650.0,2606.0,17.0,1.0,1046.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg,1.0,784.0,2057.0,36.0,1.0,1442.0
id_MzQ6QUI6OTU6NzE6OTY6MDA,7.0,429.0,3297.0,0.0,0.0,594.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ,11.0,436.0,2445.0,0.0,0.0,1439.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M,2.0,386.0,1223.0,171.0,0.0,2540.0


In [210]:
kim2

act_label,0.0,1.0,2.0,3.0,4.0,new_0
deviceId,,,,,,
id_MzA6QUU6QTQ6RTI6RTA6NEM,13.0,843.0,664.0,20.0,0.0,4161.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ,3.0,175.0,236.0,3.0,0.0,4384.0
id_MzA6QUU6QTQ6RTM6RTA6M0M,2.0,275.0,1042.0,174.0,2.0,5263.0
id_MzA6QUU6QTQ6RTM6RTA6MzA,9.0,613.0,2647.0,14.0,0.0,6368.0
id_MzQ6QUI6OTU6NzE6OTU6NzA,3.0,650.0,2606.0,17.0,1.0,6294.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg,1.0,784.0,2057.0,36.0,1.0,5630.0
id_MzQ6QUI6OTU6NzE6OTY6MDA,7.0,429.0,3297.0,0.0,0.0,7188.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ,11.0,436.0,2445.0,0.0,0.0,6329.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M,2.0,386.0,1223.0,171.0,0.0,5328.0


In [211]:
kim2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, id_MzA6QUU6QTQ6RTI6RTA6NEM to id_MzQ6QUI6OTU6NzE6QTU6MTQ
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     29 non-null     float64
 1   1.0     29 non-null     float64
 2   2.0     29 non-null     float64
 3   3.0     29 non-null     float64
 4   4.0     29 non-null     float64
 5   new_0   29 non-null     float64
dtypes: float64(6)
memory usage: 2.6+ KB


In [212]:
kim2.loc[:, [0]]

act_label,0.0
deviceId,
id_MzA6QUU6QTQ6RTI6RTA6NEM,13.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ,3.0
id_MzA6QUU6QTQ6RTM6RTA6M0M,2.0
id_MzA6QUU6QTQ6RTM6RTA6MzA,9.0
id_MzQ6QUI6OTU6NzE6OTU6NzA,3.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg,1.0
id_MzQ6QUI6OTU6NzE6OTY6MDA,7.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ,11.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M,2.0


In [218]:
kim2.to_csv('ch10/kim_stacked.csv')

In [49]:
reshaped = kim.pivot('deviceId', 'act_label')
reshaped

C:\Users\sunin\AppData\Local\Temp\ipykernel_49880\869604539.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  reshaped = kim.pivot('deviceId', 'act_label')


new_cnt                                    
act_label                      0.0     1.0     2.0     3.0    4.0  5.0
deviceId                                                              
id_MzA6QUU6QTQ6RTI6RTA6NEM    13.0   843.0   432.0   232.0   20.0  0.0
id_MzA6QUU6QTQ6RTM6OTc6RjQ     3.0   175.0   172.0    64.0    3.0  0.0
id_MzA6QUU6QTQ6RTM6RTA6M0M     2.0   275.0   375.0   667.0  174.0  2.0
id_MzA6QUU6QTQ6RTM6RTA6MzA     9.0   613.0  1300.0  1347.0   14.0  0.0
id_MzQ6QUI6OTU6NzE6OTU6NzA     3.0   650.0  1394.0  1212.0   17.0  1.0
id_MzQ6QUI6OTU6NzE6OTU6Rjg     1.0   784.0  1082.0   975.0   36.0  1.0
id_MzQ6QUI6OTU6NzE6OTY6MDA     7.0   429.0  1003.0  2294.0    0.0  0.0
id_MzQ6QUI6OTU6NzE6OTY6MDQ    11.0   436.0   639.0  1806.0    0.0  0.0
id_MzQ6QUI6OTU6NzE6OTY6Q0M     2.0   386.0   544.0   679.0  171.0  0.0
id_MzQ6QUI6OTU6NzE6OTY6RTQ    17.0   751.0   704.0   373.0    0.0  0.0
id_MzQ6QUI6OTU6NzE6OTc6RDQ     3.0   708.0   731.0   230.0   10.0  0.0
id_MzQ6QUI6OTU6NzE6OTk6MDg    18.0   701.0  1266.0   697.0   16.0  3.0
id_MzQ6QUI6OTU6NzE6OUI6MTA    16.0   648.0   952.0   593.0    7.0  0.0
id_MzQ6QUI6OTU6NzE6OUM6NDA     0.0   408.0  1047.0  1134.0   51.0  1.0
id_MzQ6QUI6OTU6NzE6OUM6OUM     5.0   744.0  1075.0  1187.0  138.0  4.0
id_MzQ6QUI6OTU6NzE6OUM6Qzg     1.0   569.0  1532.0   777.0  109.0  6.0
id_MzQ6QUI6OTU6NzE6OUM6REM     1.0   474.0  1236.0   529.0   66.0  1.0
id_MzQ6QUI6OTU6NzE6OUQ6Mzg    12.0  1224.0  1071.0   410.0   26.0  2.0
id_MzQ6QUI6OTU6NzE6OUQ6NEM    14.0   660.0   885.0   527.0   71.0  4.0
id_MzQ6QUI6OTU6NzE6OUU6MzQ     2.0   511.0  1482.0   976.0   98.0  0.0
id_MzQ6QUI6OTU6NzE6OUU6QTg     1.0   523.0  1336.0   858.0  108.0  1.0
id_MzQ6QUI6OTU6NzE6OUU6RDA    15.0   772.0  1036.0   492.0   36.0  2.0
id_MzQ6QUI6OTU6NzE6QTA6ODg     0.0   532.0  1270.0  1109.0   32.0  1.0
id_MzQ6QUI6OTU6NzE6QTE6MDA  1874.0   854.0   957.0   573.0   62.0  3.0
id_MzQ6QUI6OTU6NzE6QTQ6Mjg    19.0   655.0   763.0   378.0  438.0  6.0
id_MzQ6QUI6OTU6NzE6QTQ6Nzg     6.0    95.0   202.0  1649.0    0.0  0.0
id_MzQ6QUI6OTU6NzE6QTQ6ODg     0.0    80.0    62.0    27.0    1.0  0.0
id_MzQ6QUI6OTU6NzE6QTQ6RjQ     1.0   456.0  1554.0  1364.0   55.0  0.0
id_MzQ6QUI6OTU6NzE6QTU6MTQ     2.0   671.0  1164.0   989.0   60.0  1.0

In [51]:
# reshaped[ reshaped['act_label'] == '0']
reshaped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, id_MzA6QUU6QTQ6RTI6RTA6NEM to id_MzQ6QUI6OTU6NzE6QTU6MTQ
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (new_cnt, 0.0)  29 non-null     float64
 1   (new_cnt, 1.0)  29 non-null     float64
 2   (new_cnt, 2.0)  29 non-null     float64
 3   (new_cnt, 3.0)  29 non-null     float64
 4   (new_cnt, 4.0)  29 non-null     float64
 5   (new_cnt, 5.0)  29 non-null     float64
dtypes: float64(6)
memory usage: 1.6+ KB


In [79]:
reshaped = reshaped.reset_index()

In [91]:
reshaped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (deviceId, )    29 non-null     object 
 1   (new_cnt, 0.0)  29 non-null     float64
 2   (new_cnt, 1.0)  29 non-null     float64
 3   (new_cnt, 2.0)  29 non-null     float64
 4   (new_cnt, 3.0)  29 non-null     float64
 5   (new_cnt, 4.0)  29 non-null     float64
 6   (new_cnt, 5.0)  29 non-null     float64
 7   (new_0, )       29 non-null     float64
dtypes: float64(7), object(1)
memory usage: 1.9+ KB


In [98]:
reshaped

deviceId new_cnt                                 \
act_label                                 0.0     1.0     2.0     3.0    4.0   
0          id_MzA6QUU6QTQ6RTI6RTA6NEM    13.0   843.0   432.0   232.0   20.0   
1          id_MzA6QUU6QTQ6RTM6OTc6RjQ     3.0   175.0   172.0    64.0    3.0   
2          id_MzA6QUU6QTQ6RTM6RTA6M0M     2.0   275.0   375.0   667.0  174.0   
3          id_MzA6QUU6QTQ6RTM6RTA6MzA     9.0   613.0  1300.0  1347.0   14.0   
4          id_MzQ6QUI6OTU6NzE6OTU6NzA     3.0   650.0  1394.0  1212.0   17.0   
5          id_MzQ6QUI6OTU6NzE6OTU6Rjg     1.0   784.0  1082.0   975.0   36.0   
6          id_MzQ6QUI6OTU6NzE6OTY6MDA     7.0   429.0  1003.0  2294.0    0.0   
7          id_MzQ6QUI6OTU6NzE6OTY6MDQ    11.0   436.0   639.0  1806.0    0.0   
8          id_MzQ6QUI6OTU6NzE6OTY6Q0M     2.0   386.0   544.0   679.0  171.0   
9          id_MzQ6QUI6OTU6NzE6OTY6RTQ    17.0   751.0   704.0   373.0    0.0   
10         id_MzQ6QUI6OTU6NzE6OTc6RDQ     3.0   708.0   731.0   230.0   10.0   
11         id_MzQ6QUI6OTU6NzE6OTk6MDg    18.0   701.0  1266.0   697.0   16.0   
12         id_MzQ6QUI6OTU6NzE6OUI6MTA    16.0   648.0   952.0   593.0    7.0   
13         id_MzQ6QUI6OTU6NzE6OUM6NDA     0.0   408.0  1047.0  1134.0   51.0   
14         id_MzQ6QUI6OTU6NzE6OUM6OUM     5.0   744.0  1075.0  1187.0  138.0   
15         id_MzQ6QUI6OTU6NzE6OUM6Qzg     1.0   569.0  1532.0   777.0  109.0   
16         id_MzQ6QUI6OTU6NzE6OUM6REM     1.0   474.0  1236.0   529.0   66.0   
17         id_MzQ6QUI6OTU6NzE6OUQ6Mzg    12.0  1224.0  1071.0   410.0   26.0   
18         id_MzQ6QUI6OTU6NzE6OUQ6NEM    14.0   660.0   885.0   527.0   71.0   
19         id_MzQ6QUI6OTU6NzE6OUU6MzQ     2.0   511.0  1482.0   976.0   98.0   
20         id_MzQ6QUI6OTU6NzE6OUU6QTg     1.0   523.0  1336.0   858.0  108.0   
21         id_MzQ6QUI6OTU6NzE6OUU6RDA    15.0   772.0  1036.0   492.0   36.0   
22         id_MzQ6QUI6OTU6NzE6QTA6ODg     0.0   532.0  1270.0  1109.0   32.0   
23         id_MzQ6QUI6OTU6NzE6QTE6MDA  1874.0   854.0   957.0   573.0   62.0   
24         id_MzQ6QUI6OTU6NzE6QTQ6Mjg    19.0   655.0   763.0   378.0  438.0   
25         id_MzQ6QUI6OTU6NzE6QTQ6Nzg     6.0    95.0   202.0  1649.0    0.0   
26         id_MzQ6QUI6OTU6NzE6QTQ6ODg     0.0    80.0    62.0    27.0    1.0   
27         id_MzQ6QUI6OTU6NzE6QTQ6RjQ     1.0   456.0  1554.0  1364.0   55.0   
28         id_MzQ6QUI6OTU6NzE6QTU6MTQ     2.0   671.0  1164.0   989.0   60.0   

                 new_0  
act_label  5.0          
0          0.0  4161.0  
1          0.0  4384.0  
2          2.0  5263.0  
3          0.0  6368.0  
4          1.0  6294.0  
5          1.0  5630.0  
6          0.0  7188.0  
7          0.0  6329.0  
8          0.0  5328.0  
9          0.0  4646.0  
10         0.0  4583.0  
11         3.0  5601.0  
12         0.0  5224.0  
13         1.0  6145.0  
14         4.0  5980.0  
15         6.0  6175.0  
16         1.0  5678.0  
17         2.0  4605.0  
18         4.0  5147.0  
19         0.0  6365.0  
20         1.0  6100.0  
21         2.0  5114.0  
22         1.0  6200.0  
23         3.0  5061.0  
24         6.0  5250.0  
25         0.0  6076.0  
26         0.0  4330.0  
27         0.0  6837.0  
28         1.0  5863.0

In [101]:
reshaped['new_cnt'].iloc[0].iloc[0]

13.0

In [88]:
reshaped.apply(lambda row : print(row.iloc[2]), axis=1 )

843.0
175.0
275.0
613.0
650.0
784.0
429.0
436.0
386.0
751.0
708.0
701.0
648.0
408.0
744.0
569.0
474.0
1224.0
660.0
511.0
523.0
772.0
532.0
854.0
655.0
95.0
80.0
456.0
671.0


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
dtype: object

In [89]:
reshaped['new_0'] = reshaped.apply(lambda row : (4320  - row.iloc[2] + row.iloc[3] + row.iloc[4]+ row.iloc[5] + row.iloc[6]) , axis=1 )

In [90]:
reshaped

deviceId new_cnt                                 \
act_label                                 0.0     1.0     2.0     3.0    4.0   
0          id_MzA6QUU6QTQ6RTI6RTA6NEM    13.0   843.0   432.0   232.0   20.0   
1          id_MzA6QUU6QTQ6RTM6OTc6RjQ     3.0   175.0   172.0    64.0    3.0   
2          id_MzA6QUU6QTQ6RTM6RTA6M0M     2.0   275.0   375.0   667.0  174.0   
3          id_MzA6QUU6QTQ6RTM6RTA6MzA     9.0   613.0  1300.0  1347.0   14.0   
4          id_MzQ6QUI6OTU6NzE6OTU6NzA     3.0   650.0  1394.0  1212.0   17.0   
5          id_MzQ6QUI6OTU6NzE6OTU6Rjg     1.0   784.0  1082.0   975.0   36.0   
6          id_MzQ6QUI6OTU6NzE6OTY6MDA     7.0   429.0  1003.0  2294.0    0.0   
7          id_MzQ6QUI6OTU6NzE6OTY6MDQ    11.0   436.0   639.0  1806.0    0.0   
8          id_MzQ6QUI6OTU6NzE6OTY6Q0M     2.0   386.0   544.0   679.0  171.0   
9          id_MzQ6QUI6OTU6NzE6OTY6RTQ    17.0   751.0   704.0   373.0    0.0   
10         id_MzQ6QUI6OTU6NzE6OTc6RDQ     3.0   708.0   731.0   230.0   10.0   
11         id_MzQ6QUI6OTU6NzE6OTk6MDg    18.0   701.0  1266.0   697.0   16.0   
12         id_MzQ6QUI6OTU6NzE6OUI6MTA    16.0   648.0   952.0   593.0    7.0   
13         id_MzQ6QUI6OTU6NzE6OUM6NDA     0.0   408.0  1047.0  1134.0   51.0   
14         id_MzQ6QUI6OTU6NzE6OUM6OUM     5.0   744.0  1075.0  1187.0  138.0   
15         id_MzQ6QUI6OTU6NzE6OUM6Qzg     1.0   569.0  1532.0   777.0  109.0   
16         id_MzQ6QUI6OTU6NzE6OUM6REM     1.0   474.0  1236.0   529.0   66.0   
17         id_MzQ6QUI6OTU6NzE6OUQ6Mzg    12.0  1224.0  1071.0   410.0   26.0   
18         id_MzQ6QUI6OTU6NzE6OUQ6NEM    14.0   660.0   885.0   527.0   71.0   
19         id_MzQ6QUI6OTU6NzE6OUU6MzQ     2.0   511.0  1482.0   976.0   98.0   
20         id_MzQ6QUI6OTU6NzE6OUU6QTg     1.0   523.0  1336.0   858.0  108.0   
21         id_MzQ6QUI6OTU6NzE6OUU6RDA    15.0   772.0  1036.0   492.0   36.0   
22         id_MzQ6QUI6OTU6NzE6QTA6ODg     0.0   532.0  1270.0  1109.0   32.0   
23         id_MzQ6QUI6OTU6NzE6QTE6MDA  1874.0   854.0   957.0   573.0   62.0   
24         id_MzQ6QUI6OTU6NzE6QTQ6Mjg    19.0   655.0   763.0   378.0  438.0   
25         id_MzQ6QUI6OTU6NzE6QTQ6Nzg     6.0    95.0   202.0  1649.0    0.0   
26         id_MzQ6QUI6OTU6NzE6QTQ6ODg     0.0    80.0    62.0    27.0    1.0   
27         id_MzQ6QUI6OTU6NzE6QTQ6RjQ     1.0   456.0  1554.0  1364.0   55.0   
28         id_MzQ6QUI6OTU6NzE6QTU6MTQ     2.0   671.0  1164.0   989.0   60.0   

                 new_0  
act_label  5.0          
0          0.0  4161.0  
1          0.0  4384.0  
2          2.0  5263.0  
3          0.0  6368.0  
4          1.0  6294.0  
5          1.0  5630.0  
6          0.0  7188.0  
7          0.0  6329.0  
8          0.0  5328.0  
9          0.0  4646.0  
10         0.0  4583.0  
11         3.0  5601.0  
12         0.0  5224.0  
13         1.0  6145.0  
14         4.0  5980.0  
15         6.0  6175.0  
16         1.0  5678.0  
17         2.0  4605.0  
18         4.0  5147.0  
19         0.0  6365.0  
20         1.0  6100.0  
21         2.0  5114.0  
22         1.0  6200.0  
23         3.0  5061.0  
24         6.0  5250.0  
25         0.0  6076.0  
26         0.0  4330.0  
27         0.0  6837.0  
28         1.0  5863.0

'id_MzA6QUU6QTQ6RTM6OTc6RjQ'

In [74]:
reshaped.apply(lambda row : print(row['0.0']) , axis=1)

KeyError: '0.0'

In [ ]:
# reshaped.apply(lambda )

In [50]:
unstacked = reshaped.set_index(['deviceid','act_label']).unstack('new_cnt')

KeyError: "None of ['deviceid', 'act_label'] are in the columns"

In [21]:
kim = pd.merge(kim, new2, on=['deviceId', 'act_label'], how="left")
kim

ValueError: 'act_label' is both an index level and a column label, which is ambiguous.